# PHASM501 Coursework 1 - Diffusion Solver $\hspace{4cm}$ Liam Eloie

## Part 1 - Explanation

This notebook provides a solution to the diffusion equation,

$$-\nabla \cdot [\sigma(x) \nabla u(x)] = f(x), \text{in}\hspace{0.1cm} \Omega$$

Subject to the boundary condition,
$$u(x) = 0, \text{on}\hspace{0.1cm}\partial \Omega$$

Where $\Omega$ is a given domain in two dimensions, $f$ is a given function in $\Omega$, $\sigma$ is a scalar field that is defined over $\Omega$ and $u$ is the wanted solution.


By discretising the problem by multiplying with a test function, $v$, that satisfy the zero boundary conditions, applying Green's representation theorem, and approximating the solution of $u$ by expanding it into a linear combination of P1 basis functions, the problem boils down to solving the following system,

$$\int_\Omega [\sigma(\mathbf{x})\nabla \psi_i(\mathbf{x})] \cdot \nabla\psi_j(\mathbf{x}) d\mathbf{x} = \int_\Omega f(\mathbf{x})\psi_i(\mathbf{x}) d\mathbf{x}$$

Where the P1 basis functions are,
 $$\psi_i(\mathbf{x}) = \begin{cases} 
      1 - x - y & i = 0 \\
      x & i = 1 \\
      y & i = 2 
   \end{cases}
$$

We will be working with triangular grids. To make the integration over a triangle a lot easier, we will be transformation each triangle in the grid into a right angle triangle with corners lying on the points, $(0, 0)$, $(1, 0)$, and $(0, 1)$. This simplies the P1 basis functions considerably.

In order to make this transformation of basis, we must apply the following transformation,

$$\mathbf{x} = \mathbf{x}_0 \psi_0(\mathbf{X}) + \mathbf{x}_1 \psi_1(\mathbf{X}) + \mathbf{x}_2 \psi_2(\mathbf{X})$$

Where $\mathbf{X}$ is a point on the reference triangle.

This allows us to define the jacobian on the system, 

$$J = \frac{\partial x}{\partial X}\frac{\partial y}{\partial Y} - \frac{\partial y}{\partial X}\frac{\partial x}{\partial Y}$$

such that the transformation becomes,

$$g(\mathbf{X}) = J \mathbf{X} + \mathbf{x}_0$$

Using this information, we can transform our integration over the triangles in the grid to an integration over reference triangles to make use of the simple nature of the P1 basis functions,

$$\int_R [\sigma(g(\mathbf{X})) J^{-T} \nabla\psi_i(\mathbf{X})] \cdot J^{-T} \nabla\psi_j(\mathbf{X}) | \mathrm{det}(J) | d\mathbf{X} = \int_R f(g(\mathbf{X}))\psi_i(g(\mathbf{X})) | \mathrm{det}(J) | d\mathbf{X}$$


We can turn this into a matrix equation by defining 

$$a(u, v) = \int_\Omega [\sigma(\mathbf{x})\nabla u(\mathbf{x})] \cdot \nabla v(\mathbf{x}) d\mathbf{x}$$
$$l(v) = \int_\Omega f(\mathbf{x}) v(\mathbf{x}) d\mathbf{x}$$

and it is possible to show
$$A\mathbf{u} = \mathbf{b}$$

where

$$A_{i,j} = a(\psi_j(\mathbf{X}), \psi_i(\mathbf{X}))$$

$$b_i = l(\psi_i(\mathbf{X}))$$

The following codes solves this system by implementing that matrix equation and solving for $u(\mathbf{x})$. 

$$u(\mathbf{x}) = A^{-1}\mathbf{b}$$

To compute $a(u,v)$ and $l(v)$, we must perform an integration over triangles, T. To do this will shall use the quadrature rule. 

$$\int_T f(\mathbf{x}) d\mathbf{x} \approx \frac{1}{2} A_T \sum_{i=1}^{N_g} \omega_i F(P(\mathbf{X}), Q(\mathbf{X}))$$

Where the sum is over all the quadrature points, $\omega_i$ are the weights of each point which sum to $1$, and $A_T$ is the area of each triangle in T. P and Q are the transformation from reference to global triangles.

## Part 2 - Code and Graphical Visualisations

First, we want to define what our scalar field $\sigma(\mathbf{x})$ and our function $f(\mathbf{x})$ will be.

In [1]:
import numpy as np 

def func(x, y):
    return 1

def sigma(x, y):
    return 1

Next, we would like to choose the VTK file name for the grid we would like to solve the diffusion equation on. 

A VTK file supplies all points and cells that makes up a grid. Each cell is made up of points in the grid and have a specified cell type (1 for point, 3 for line segment, 5 for triangle etc.)


In [2]:
grid_dir = './example_grids/'
grid_file_name = "Small_Square_Triangle_Grid"

When running the solver, we need to specify the order of the underlying quadrature rule we will be using to perform our integration over triangle elements.

In [3]:
integration_order = 4

Finally, we will insert these functions and variables into the global solver. This global solver solves our system and writes a VTK file containing details of all the points, cells, and the solution on each point.

In [4]:
from source import solver

solver.solve(grid_dir + grid_file_name, func, sigma, integration_order)

Importing VTK file grid data.
Setting up grid.
Constructing system of assembly matrix and force vector.
Solving for the solution.
Exporting solution to a VTK file located at: ./example_grids/Small_Square_Triangle_Grid_solution.vtk


Using this VTK output file for the grid and solution, we can use the Paraview software to give graphical visualisations of our solution.

In [5]:
from IPython.display import HTML, display

display(HTML("<table><tr><td><img src='img/diffusion_constant_bad_grid.png'></td><td><img src='img/diffusion_constant_bad.png'></td></tr></table>"))

,


If we increase the number of points and elements in a grid, the solver will return a more accurate solution. 

In [6]:
grid_file_name = "Big_Square_Triangle_Grid"

solver.solve(grid_dir + grid_file_name, func, sigma, integration_order)

Importing VTK file grid data.
Setting up grid.
Constructing system of assembly matrix and force vector.
Solving for the solution.
Exporting solution to a VTK file located at: ./example_grids/Big_Square_Triangle_Grid_solution.vtk


In [7]:
display(HTML("<table><tr><td><img src='img/diffusion_constant_f_grid.png'></td><td><img src='img/diffusion_constant_f.png'></td></tr></table>"))

,


On the left is the solution with the overlayed on top. On the right is the solution without the grid.

Since our diffusion solver is general, we can solve for more intricate systems over different shaped grids made up of a triangle mesh. Below are some systems I found to be interesting.

In [8]:
def func(x, y):
    if x > 0.15 and x < 0.25 and y > 0.15 and y < 0.25:
        return 1
    
    elif x > 0.75 and x < 0.85 and y > 0.15 and y < 0.25:
        return 1
    
    elif x > 0.15 and x < 0.25 and y > 0.75 and y < 0.85:
        return 1
    
    elif x > 0.75 and x < 0.85 and y > 0.75 and y < 0.85:
        return 1
    
    else: 
        return 0

def sigma(x, y):
    return 1

grid_file_name = "Big_Square_Triangle_Grid"

solver.solve(grid_dir + grid_file_name, func, sigma, integration_order)

Importing VTK file grid data.
Setting up grid.
Constructing system of assembly matrix and force vector.
Solving for the solution.
Exporting solution to a VTK file located at: ./example_grids/Big_Square_Triangle_Grid_solution.vtk


In [9]:
display(HTML("<table><tr><td><img src='img/interesting_solution_1_grid.png'></td><td><img src='img/interesting_solution_1.png'></td></tr></table>"))

,


This solution represents the diffusion when the function on the right-hand side of the equation is instantiated at four distinct points on the grid. This is the kind of pattern we would expect since the instantiation, grid, and boundary conditions are symmetric.

We are not only limited to square grid. The next example demonstrates the versatility of our solver.

In [10]:
grid_file_name = "Big_Weird_Grid"

def func(x, y):
    return 1

solver.solve(grid_dir + grid_file_name, func, sigma, integration_order)

Importing VTK file grid data.
Setting up grid.
Constructing system of assembly matrix and force vector.
Solving for the solution.
Exporting solution to a VTK file located at: ./example_grids/Big_Weird_Grid_solution.vtk


In [11]:
display(HTML("<table><tr><td><img src='img/diffusion_weird_solution_grid.png'></td><td><img src='img/diffusion_weird_solution.png'></td></tr></table>"))

,
